In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# csv_file = os.path.abspath('')
data = np.genfromtxt('./dataR2.csv', delimiter=',', skip_header=True)
# mieszam sobie zbiór, bo... bo mogę
np.random.shuffle(data)
# height to de facto liczba badań
height, _ = data.shape
# liczba zbiorów testowych, liczba zbiorów treningowych
# potem się okaże, że wystarczy jedno z nich, ale nie chce mi się już używać korektora
n_test, n_train = round(0.2*height), height-round(0.2*height)
# sprawdzam, czy nie potknąłem się o zaokrąglenie, ale pewnie jakby wziąć pod uwagę dokładność numeryczną, to poniższy krok prawdopodobnie nie ma sensu
# (sztuka dla sztuki, jak paw dumnie prezentuje swój ogon, tak jak dumnie prezentuję znajomość niesamowicie skomplikowanej składni języka Python)
# <owacje>
assert n_test+n_train==height

In [3]:
# remove broken data (needles if skip_header=True)
filtered = []
for row in data:
    if np.isnan(np.sum(row)):
        print(row)

In [20]:
train_set = data[:n_train,:]
train_x, train_y = train_set[:,:-1], train_set[:,-1:].squeeze()
print(train_x.shape, train_y.shape)
test_set=data[n_train:,:]
test_x, test_y = test_set[:,:-1], test_set[:,-1:].squeeze()
print(train_set.shape, test_set.shape, n_train, n_test)
total_x, total_y = data[:,:-1], data[:,-1:].squeeze()
# nie ma potrzeby normalizacji train_y i test_y do {0,1}, ponieważ macierz predykcji wskazuje na prawdopodobieństwo wystąpienia kolejnych klas
# (w naszym przypadku to 1 i 2)

(93, 9) (93,)
(93, 10) (23, 10) 93 23


In [5]:
model = LogisticRegression(solver='liblinear', random_state=0, ).fit(train_x, train_y)
# podobno liblinear is good for small datasets, a ja nie mam zamiaru się z tym kłucić

In [6]:
print(model.classes_)
# ważne!
# w naszym przypadku są dwie klasy (liczba kolumn w macierzy predict_proba)
# model.predict bierze pod uwagę macierz n x 2, gdzie pierwsza kolumna to prawdopodobieństwo pierwszej klasy (1),
# a druga kolumna to prawd drugiej klasy (2). Czyli (wg mnie) próg odcięcia tutaj wynosi 0.5
pred_y = model.predict(test_x)
cls_report = classification_report(test_y, pred_y, output_dict=False)
cls_report_dict = classification_report(test_y, pred_y, output_dict=True)
print(cls_report)
trafnosc_predykcji = cls_report_dict.keys()
trafnosc_decyzji = round(cls_report_dict['macro avg']['f1-score'], 3)
# podobno recall to jest czułość
czulosc = round(cls_report_dict['macro avg']['recall'], 3)
print('***')
print(f'Odp.: Trafność decyzji wynosi {trafnosc_decyzji}, a czułość modelu {czulosc}.')

[1. 2.]
              precision    recall  f1-score   support

         1.0       0.71      0.92      0.80        13
         2.0       0.83      0.50      0.62        10

    accuracy                           0.74        23
   macro avg       0.77      0.71      0.71        23
weighted avg       0.76      0.74      0.72        23

***
Odp.: Trafność decyzji wynosi 0.713, a czułość modelu 0.712.


In [7]:
# **********
# podpunkt B
# **********

COST_TP = 48_477
COST_TN = 0
COST_FP = 751
COST_FN = 89_463

In [23]:
from scipy.optimize import minimize
from sklearn.metrics import confusion_matrix

def get_py(model, p, t):
    return [model.classes_[1] if pi[1] > t else model.classes_[0] for pi in p]

def optimize(model, x, y):
    # FUNKCJA OPTYMALIZUJĄCA (podpunkt b)
    # dostajemy nauczony model oraz testowe x i y (na podstawie których będziemy optymalizować)
    p = model.predict_proba(x)
    t0 = 0.5
    def fun(t):
        # print(t)
        # poniżej uzależniamy predykcję od punktu odcięcia t
        py = get_py(model, p, t)
        cm = confusion_matrix(y, py)
        tn, fp, fn, tp = cm.ravel()
        return tn * COST_TN + fp * COST_FP + fn * COST_FN + tp * COST_TP
        # print(py)
        # print(model.predict(x))
    # cm = confusion_matrix(y, py)
    # print(cm)
    res = minimize(fun, t0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
    return res.x[0]

# optimize(model, test_x, test_y)
# print(total_x.shape, total_y.shape)
# optimize(model, total_x, total_y)
# print(test_y)
# print(model.predict(test_x))

In [28]:
print('--- tylko dla danych testowych (podpunkt c) ---')
t_opt = round(optimize(model, test_x, test_y), 3)
print(f'optymalny punkt odciecia = {t_opt}')
opt_y = get_py(model, model.predict_proba(test_x), t_opt)
print(opt_y)
cls_report = classification_report(test_y, opt_y, output_dict=False)
cls_report_dict = classification_report(test_y, opt_y, output_dict=True)
print(cls_report)
trafnosc_predykcji = cls_report_dict.keys()
trafnosc_decyzji = round(cls_report_dict['macro avg']['f1-score'], 3)
# podobno recall to jest czułość
czulosc = round(cls_report_dict['macro avg']['recall'], 3)
print('***')
print(f'Odp.: Trafność decyzji wynosi {trafnosc_decyzji}, a czułość modelu {czulosc}.')

--- tylko dla danych testowych ---
Optimization terminated successfully.
         Current function value: 650216.000000
         Iterations: 24
         Function evaluations: 69
optymalny punkt odciecia = 0.475
[2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0]
              precision    recall  f1-score   support

         1.0       0.73      0.85      0.79        13
         2.0       0.75      0.60      0.67        10

    accuracy                           0.74        23
   macro avg       0.74      0.72      0.73        23
weighted avg       0.74      0.74      0.73        23

***
Odp.: Trafność decyzji wynosi 0.726, a czułość modelu 0.723.


In [33]:
print('--- dla całego zbioru danych (podpunkt c) ---')
t_opt = round(optimize(model, total_x, total_y), 3)
print(f'optymalny punkt odciecia = {t_opt}')
opt_y = get_py(model, model.predict_proba(total_x), t_opt)
print(opt_y)
cls_report = classification_report(total_y, opt_y, output_dict=False)
cls_report_dict = classification_report(total_y, opt_y, output_dict=True)
print(cls_report)
trafnosc_predykcji = cls_report_dict.keys()
trafnosc_decyzji = round(cls_report_dict['macro avg']['f1-score'], 3)
# podobno recall to jest czułość
czulosc = round(cls_report_dict['macro avg']['recall'], 3)
print('***')
print(f'Odp.: Trafność decyzji wynosi {trafnosc_decyzji}, a czułość modelu {czulosc}.')

--- dla całego zbioru danych ---
Optimization terminated successfully.
         Current function value: 3137074.000000
         Iterations: 32
         Function evaluations: 86
optymalny punkt odciecia = 0.1
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0]
              precision    recall  f1-score   support

         1.0       1.00      0.12      0.21        52
         2.0       0.58      1.00      0.74        64

    accuracy                           0.60    

In [35]:
print("""
moja obserwacja:
Finansowo optymalny punkt odcięcia to 0.1. Pomimo, że model wówczas jest mało precyzyjny, to ma to sens, gdyż lepiej (finansowo) jest więcej osób sklasyfikować jako chore i ew poświęcić 751 kosztu niż błędnie klasyfikować więcej osób jako zdrowe i poświęcać potem dwukrony koszt leczenie. Zdaje się to całkiem intuicyjne.
---
wytłumaczenie:
Powyżej optymalizowałem tylko dla danych testowych i widać, że punkt odcięcia poszedł trochę w dół, ale porządna optymalizacja wyszła dopiero dla wszystkich danych.
""")


moja obserwacja:
Finansowo optymalny punkt odcięcia to 0.1. Pomimo, że model wówczas jest mało precyzyjny, to ma to sens, gdyż lepiej (finansowo) jest więcej osób sklasyfikować jako chore i ew poświęcić 751 kosztu niż błędnie klasyfikować więcej osób jako zdrowe i poświęcać potem dwukrony koszt leczenie. Zdaje się to całkiem intuicyjne.
---
wytłumaczenie:
Powyżej optymalizowałem tylko dla danych testowych i widać, że punkt odcięcia poszedł trochę w dół, ale porządna optymalizacja wyszła dopiero dla wszystkich danych.



In [43]:
"""
dla porównania zmodyfikowaliśmy funkcję optimize tak, żeby przyjmowała koszty=0 dla dobrych predykcji i 1 dla złych
(chodzi o to, żeby tylko w przypadku złej decyzji trzeba było płacić)
teraz widać, że punkt odcięcia zmienia się tak, żeby model był jak najbardziej skuteczny
"""

def optimize2(model, x, y):
    # FUNKCJA OPTYMALIZUJĄCA (podpunkt b)
    # dostajemy nauczony model oraz testowe x i y (na podstawie których będziemy optymalizować)
    p = model.predict_proba(x)
    t0 = 0.5
    def fun(t):
        # poniżej uzależniamy predykcję od punktu odcięcia t
        py = get_py(model, p, t)
        cm = confusion_matrix(y, py)
        tn, fp, fn, tp = cm.ravel()
        return tn * 0 + fp * 1 + fn * 1 + tp * 0
        # print(py)
        # print(model.predict(x))
    # cm = confusion_matrix(y, py)
    # print(cm)
    res = minimize(fun, t0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
    return res.x[0]

print('--- dla całego zbioru danych (podpunkt c) ---')
t_opt = round(optimize2(model, total_x, total_y), 3)
print(f'optymalny punkt odciecia = {t_opt}')
opt_y = get_py(model, model.predict_proba(total_x), t_opt)
print(opt_y)
cls_report = classification_report(total_y, opt_y, output_dict=False)
cls_report_dict = classification_report(total_y, opt_y, output_dict=True)
print(cls_report)
trafnosc_predykcji = cls_report_dict.keys()
trafnosc_decyzji = round(cls_report_dict['macro avg']['f1-score'], 3)
# podobno recall to jest czułość
czulosc = round(cls_report_dict['macro avg']['recall'], 3)
print('***')
print(f'Odp.: Trafność decyzji wynosi {trafnosc_decyzji}, a czułość modelu {czulosc}.')

--- dla całego zbioru danych (podpunkt c) ---
Optimization terminated successfully.
         Current function value: 24.000000
         Iterations: 24
         Function evaluations: 68
optymalny punkt odciecia = 0.497
[1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0]
              precision    recall  f1-score   support

         1.0       0.75      0.81      0.78        52
         2.0       0.83      0.78      0.81        64

    accuracy                         